This file assumes you have run Book 3 and that the AGM database in MongoDB is initialized. Please ensure that you are using the MongoDB supported cluster.

The point of this script is to walk through how to access MongoDB and visualize the routes data

In [1]:
# Run this to install necessary packages and restart the kernel for osmnx import error to be resolved

# !pip install geopandas
# !pip install folium
# !pip install geopy
# !pip install taxicab
# !pip install osmnx==1.9.1
# !pip install --upgrade osmnx matplotlib numpy

In [2]:
import pymongo
import json
import pprint

import pandas as pd
from IPython.display import display

import warnings

warnings.simplefilter("ignore", category=FutureWarning)

import geopandas as gpd
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium.features import DivIcon
import requests 
import numpy as np
import random
from geopy.geocoders import Nominatim
from IPython.display import Image, IFrame 
from IPython.core.display import HTML 
from pandas import json_normalize
%matplotlib inline
import osmnx as ox
from osmnx import plot_route_folium
import networkx as nx
import matplotlib
import matplotlib as plt
import taxicab as tc
from IPython.display import clear_output
import csv

/opt/conda/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Lets initialize our client for MongoDB and pull the data

In [3]:
mongo = pymongo.MongoClient("mongodb://mongo:27017/")

In [4]:
db_list = mongo.database_names()

db_list

/tmp/ipykernel_211/3002110245.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  db_list = mongo.database_names()


['AGM', 'admin', 'config', 'local']

In [5]:
my_client = mongo["AGM"]

db_routes = my_client["AGM_routes"]
db_store_info = my_client["AGM_store_info"]

Lets verify our data is coming through correctly

In [6]:
# Find all documents in the collection
cursor = db_routes.find()

# Iterate over the cursor and print each document
for document in cursor:
    print(document)

{'_id': ObjectId('65fd00e395522cd3cb5a2359'), 'Alameda': {'nodes': ['PhantomMotors', 'BusStop5077328SanLeandroBart', 'CaliforniaExpressMotors', 'HaywardBARTGarage', 'MensWearhouse', 'RedApplePizza', 'Up2SpeedSportsTherapy', 'SimbolMaterials', 'Logitech', 'MyUnionCityPlumberHero', 'SpeeDeeOilChangeandTuneUp', 'EnvieInteractive', 'UniversityUnion', 'SpeedeeOilChangeTuneUp', 'SafeAmericaCreditUnion', 'TintNSound', 'DesignEnergyGroup', 'RandallE_Strauss', 'EmcorIntegratedSolutions', 'PhoPlay'], 'coordinates': ['(37.678241, -122.09081)', '(37.695409, -122.077991)', '(37.656344, -122.071653)', '(37.670639, -122.087372)', '(37.692118, -121.927759)', '(37.696829, -122.139635)', '(37.698831, -121.917708)', '(37.661614, -121.903984)', '(37.552155, -122.063208)', '(37.581319, -122.017075)', '(37.708072, -121.924819)', '(37.411732, -122.014626)', '(37.654885, -122.055788)', '(37.693297, -122.066176)', '(37.695814, -121.932341)', '(37.769199, -122.238094)', '(37.698278, -121.919474)', '(37.807695, 

In [7]:
# Find all documents in the collection
cursor = db_store_info.find()

# Iterate over the cursor and print each document
for document in cursor:
    print(document)

{'_id': ObjectId('65fd00e395522cd3cb5a235a'), 'PhantomMotors': {'name': 'PhoPlay', 'items_requested': {'Teriyaki Chicken': 23, 'Eggplant Lasagna': 83, 'Curry Chicken': 16, 'Brocolli Stir Fry': 53, 'Pistachio Salmon': 95, 'Spinach Orzo': 59, 'Chicken Salad': 65, 'Tilapia Piccata': 87}, 'delivered_tag': False, 'time of delivery': '00:00:00'}, 'BusStop5077328SanLeandroBart': {'name': 'PhoPlay', 'items_requested': {'Teriyaki Chicken': 23, 'Eggplant Lasagna': 83, 'Curry Chicken': 16, 'Brocolli Stir Fry': 53, 'Pistachio Salmon': 95, 'Spinach Orzo': 59, 'Chicken Salad': 65, 'Tilapia Piccata': 87}, 'delivered_tag': False, 'time of delivery': '00:00:00'}, 'CaliforniaExpressMotors': {'name': 'PhoPlay', 'items_requested': {'Teriyaki Chicken': 23, 'Eggplant Lasagna': 83, 'Curry Chicken': 16, 'Brocolli Stir Fry': 53, 'Pistachio Salmon': 95, 'Spinach Orzo': 59, 'Chicken Salad': 65, 'Tilapia Piccata': 87}, 'delivered_tag': False, 'time of delivery': '00:00:00'}, 'HaywardBARTGarage': {'name': 'PhoPlay

This looks good! Next we need to work on converting the JSON into a form we can use to plot the various paths

In [76]:
# Find all documents in the collection
routes = db_routes.find()

counties = list(routes[0].keys())[1::]

nodes_list = {}
coordinates_list = {}
route_list = {}

for county in counties:
    
    print(f"\n{county}\n")
    
#     print(routes[0][county]['nodes'])
    nodes_list[county] = routes[0][county]['nodes']
    coordinates_list[county] = routes[0][county]['coordinates']
    route_list[county] = routes[0][county]['route']
    
    print(nodes_list[county])    
    print(coordinates_list[county])
    print(route_list[county])
    


Alameda

['PhantomMotors', 'BusStop5077328SanLeandroBart', 'CaliforniaExpressMotors', 'HaywardBARTGarage', 'MensWearhouse', 'RedApplePizza', 'Up2SpeedSportsTherapy', 'SimbolMaterials', 'Logitech', 'MyUnionCityPlumberHero', 'SpeeDeeOilChangeandTuneUp', 'EnvieInteractive', 'UniversityUnion', 'SpeedeeOilChangeTuneUp', 'SafeAmericaCreditUnion', 'TintNSound', 'DesignEnergyGroup', 'RandallE_Strauss', 'EmcorIntegratedSolutions', 'PhoPlay']
['(37.678241, -122.09081)', '(37.695409, -122.077991)', '(37.656344, -122.071653)', '(37.670639, -122.087372)', '(37.692118, -121.927759)', '(37.696829, -122.139635)', '(37.698831, -121.917708)', '(37.661614, -121.903984)', '(37.552155, -122.063208)', '(37.581319, -122.017075)', '(37.708072, -121.924819)', '(37.411732, -122.014626)', '(37.654885, -122.055788)', '(37.693297, -122.066176)', '(37.695814, -121.932341)', '(37.769199, -122.238094)', '(37.698278, -121.919474)', '(37.807695, -122.265135)', '(37.663021, -121.899895)', '(37.694429, -122.051597)']
[[

In [77]:
# Find all documents in the collection
routes = db_routes.find()

counties = list(routes[0].keys())[1::]

# Counties
print(counties)

['Alameda', 'San Francisco', 'Contra Costa', 'San Mateo', 'Santa Clara']


In [65]:
# Initialze the various maps we will need ahead of time to save on computing

maps = {}

for county in counties:
    maps[county] = ox.graph_from_place(f"{county}, California, USA", network_type="drive")
    
print("Done!")

Done!


In [94]:
# Lets start by initializing all maps

counties = ['San Francisco', 'Santa Clara', 'Contra Costa', 'Alameda', 'San Mateo']
coordinates = [[37.759344, -122.446452], [37.258336, -121.734320], [37.955321, -121.879418], 
               [37.674112, -121.971002], [37.466796, -122.391435]]

# Declare a Hash Table to keep track of the maps

folium_maps = {}

for i, county in enumerate(counties):
    folium_maps[county] = folium.Map(location=coordinates[i], tiles='openstreetmap', prefer_canvas=True)


In [95]:
# Auxiliary function for plotting points 

def plot_point(name, coords, map_):
    
    point = coords.strip("()").split(", ")

    point = (float(point[0]), float(point[1]))

    folium.CircleMarker(location=[point[0], point[1]],
                        radius=2,
                        weight=5,
                       color='blue',
                       label = name).add_to(map_)
    folium.map.Marker(location=[point[0], point[1]],
                  icon=DivIcon(
                      icon_size=(20,20),
                      icon_anchor=(5,14),
                      html=f'<div style="font-size: 7pt">%s</div>' % name,
                  )
                 ).add_to(map_)
# -------------------------------------------------------------------------------------------------------------
    
# Plot all features for all maps

# nodes - names
# coordinates - coordinates
# route_list - list of routes

# Plot Nodes 
for county in counties:
    
    print(county)
    
    names = nodes_list[county]
    coords = coordinates_list[county]
    
    for i in range(len(names)):
              
        plot_point(names[i], coords[i], folium_maps[county])
        
    
    folium_maps[county].fit_bounds(folium_maps[county].get_bounds())


San Francisco
Santa Clara
Contra Costa
Alameda
San Mateo


In [97]:
# Auxiliary function for animating paths

import time

def animate(names, routes, county, map_):
    
    G = maps[county]
    
    print(len(names), len(routes))
    
    for i in range(len(routes)):
        
        clear_output(wait=True)        
        print(f"Viewing Route for {county} County | Currently enroute to {names[i]} from {names[i+1]}")
        
        temp = map_
        
        display(ox.plot_route_folium(G, routes[i], route_map=temp, weight=10))
        
        time.sleep(1)
        
# -------------------------------------------------------------------------

# Plot Routes From BART to Central Node

county = "San Francisco"

names = nodes_list[county]
routes = route_list[county]

animate(names, routes, county, folium_maps[county])


Viewing Route for San Francisco County | Currently enroute to GApplianceRepair from QuickStartIntelligence


In [98]:
for i, county in enumerate(counties):
    
    print(county)
    display(folium_maps[county])

San Francisco


Santa Clara


Contra Costa


Alameda


San Mateo
